**Connect google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive /mydrive
!ls /mydrive

In [ ]:
!ls /mydrive/Ranison

 images.zip  'Train_YoloV3 .ipynb'   yolov3


In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Oct 28 13:01:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14358, done.
remote: Total 14358 (delta 0), reused 0 (delta 0), pack-reused 14358
Receiving objects: 100% (14358/14358), 13.06 MiB | 11.30 MiB/s, done.
Resolving deltas: 100% (9765/9765), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/Ranison/yolov3/"

In [ ]:
!echo "tomato" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/Ranison/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-10-28 13:04:49--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   105KB/s    in 31m 19s 

2020-10-28 13:36:09 (84.5 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!ls

3rdparty	darknet			image_yolov4.sh        README.md
backup		darknet53.conv.74	include		       results
build		DarknetConfig.cmake.in	json_mjpeg_streams.sh  scripts
build.ps1	darknet_images.py	LICENSE		       src
build.sh	darknet.py		Makefile	       video_yolov3.sh
cfg		darknet_video.py	net_cam_v3.sh	       video_yolov4.sh
cmake		data			net_cam_v4.sh
CMakeLists.txt	image_yolov3.sh		obj


In [ ]:
!unzip /mydrive/Ranison/images.zip -d data/obj

Archive:  /mydrive/Ranison/images.zip
  inflating: data/obj/images/1.jpg   
  inflating: data/obj/images/1.txt   
  inflating: data/obj/images/10.jpg  
  inflating: data/obj/images/10.txt  
  inflating: data/obj/images/100.jpg  
  inflating: data/obj/images/100.txt  
  inflating: data/obj/images/11.jpg  
  inflating: data/obj/images/11.txt  
  inflating: data/obj/images/12.jpg  
  inflating: data/obj/images/12.txt  
  inflating: data/obj/images/13.jpg  
  inflating: data/obj/images/13.txt  
  inflating: data/obj/images/14.jpg  
  inflating: data/obj/images/14.txt  
  inflating: data/obj/images/15.jpg  
  inflating: data/obj/images/15.txt  
  inflating: data/obj/images/16.jpg  
  inflating: data/obj/images/16.txt  
  inflating: data/obj/images/18.jpg  
  inflating: data/obj/images/18.txt  
  inflating: data/obj/images/19.jpg  
  inflating: data/obj/images/19.txt  
  inflating: data/obj/images/2.jpg   
  inflating: data/obj/images/2.txt   
  inflating: data/obj/images/20.jpg  
  inflatin

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/images/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.
0 0.958984 0.344666 0.082031 0.161782
15 0.731250 0.403283 0.179688 0.276671

['15', '0.731250', '0.403283', '0.179688', '0.276671']
8 data/obj/images/93.txt
0 0.731250 0.403283 0.179688 0.276671
15 0.351369 0.641011 0.130940 0.197145

['15', '0.351369', '0.641011', '0.130940', '0.197145']
9 data/obj/images/91.txt
0 0.351369 0.641011 0.130940 0.197145
15 0.493931 0.511574 0.217717 0.282407

['15', '0.493931', '0.511574', '0.217717', '0.282407']
9 data/obj/images/91.txt
0 0.493931 0.511574 0.217717 0.282407
15 0.387913 0.336613 0.150826 0.188657

['15', '0.387913', '0.336613', '0.150826', '0.188657']
9 data/obj/images/91.txt
0 0.387913 0.336613 0.150826 0.188657
15 0.343750 0.583236 0.081250 0.111372

['15', '0.343750', '0.583236', '0.081250', '0.111372']
10 data/obj/images/85.txt
0 0.343750 0.583236 0.081250 0.111372
15 0.533984 0.583822 0.097656 0.112544

['15', '0.533984', '0.583822', '0.097656', '0.112544']
10 data/obj/images/85.txt

In [ ]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)

['data/obj/images/42.jpg', 'data/obj/images/33.jpg', 'data/obj/images/100.jpg', 'data/obj/images/99.jpg', 'data/obj/images/88.jpg', 'data/obj/images/12.jpg', 'data/obj/images/77.jpg', 'data/obj/images/38.jpg', 'data/obj/images/53.jpg', 'data/obj/images/79.jpg', 'data/obj/images/18.jpg', 'data/obj/images/48.jpg', 'data/obj/images/75.jpg', 'data/obj/images/80.jpg', 'data/obj/images/26.jpg', 'data/obj/images/81.jpg', 'data/obj/images/64.jpg', 'data/obj/images/70.jpg', 'data/obj/images/62.jpg', 'data/obj/images/68.jpg', 'data/obj/images/82.jpg', 'data/obj/images/30.jpg', 'data/obj/images/35.jpg', 'data/obj/images/76.jpg', 'data/obj/images/3.jpg', 'data/obj/images/72.jpg', 'data/obj/images/85.jpg', 'data/obj/images/94.jpg', 'data/obj/images/49.jpg', 'data/obj/images/47.jpg', 'data/obj/images/93.jpg', 'data/obj/images/45.jpg', 'data/obj/images/14.jpg', 'data/obj/images/74.jpg', 'data/obj/images/7.jpg', 'data/obj/images/59.jpg', 'data/obj/images/54.jpg', 'data/obj/images/4.jpg', 'data/obj/ima

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close()

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.

 3926: 0.395697, 0.383335 avg loss, 0.001000 rate, 6.253158 seconds, 251264 images, 0.210355 hours left
Loaded: 0.000037 seconds
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.891084, GIOU: 0.889843), Class: 0.999881, Obj: 0.947501, No Obj: 0.010973, .5R: 1.000000, .75R: 0.954545, count: 22, class_loss = 0.240115, iou_loss = 0.255034, total_loss = 0.495149 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.895299, GIOU: 0.892556), Class: 0.999858, Obj: 0.978414, No Obj: 0.001304, .5R: 1.000000, .75R: 1.000000, count: 10, class_loss = 0.013015, iou_loss = 0.112292, total_loss = 0.125306 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000002, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000342, iou_loss = 0.000000, total_loss = 0.000342 
 total_bbox = 2504554, rewritten_bbox = 0.234812 % 
v3